# Convert Polar Stereographic Files to Lat/Lon Files

This notebook provides a workflow for converting polar stereographic netcdf files to lat/lon coordinates. To get the data into the proper format, we needed to do the following:
 - utilize a template file as a base for modifying netcdf files of the existing mask arrays. The template file includes all proper GIS attributes.
 - python script reads the Geotiff file and creates a netcdf file with the x,y,lat,lon coordinate dimensions and variables

Scripts were originally created by John Truesdale and updated by Teagan King.
 
To Do:
------
 - use the standard ESMF mapping procedure to go from the projected stereographic polar mask grids to our regular gridded data from CESM. Use Steve's python code and John's mapping file commands to do this

Some notes from John on methods for regridding:
-----------------------------------------------
 - used QGIS to assign a standard stereographic coordinate and create a GeoTiff file version of one of the Polar jpg plots
 - Giving a standard location to each jpeg pixel basically consisted of finding similar features at the pixel level (the tip of an island, the most inset part of a promenent bay) between our polar jpegs and a map that is already georeferenced and has known locations for those pixels. If you choose 3-10 pixels in common you can create a linear regression that will map out all the rest of the pixels on your jpeg.
 - Once the GIS application was able to calculate the transform to go from pixel to a standard coordinate system, I saved all that information in a GeoTiff file.
 - The polar projection jpegs on which the climatenet masks are drawn were created from python matplotlib and you can grab the coordinate information from matplotlib; this was checked with QGIS
 - Because the LLNL polar jpegs are a projected coordinate, the underlying unit in a stereographic projection is meters.  The x and y variables on the GeoTiff and converted netcdf file contain meter offsets of every pixel (row,col) of the ar_mask array with respect to one of the standard south pole stereographic coordinate systems.
 -   When you look at the square projected polar image you see that the longitude lines converge at the pole and latitudes are a set of nested circles.  When you are describing this grid in lat/lon coordinates it is known as a curvilinear grid where each pixel (array location) requires a unique lat/lon pair to specify its position on a regular grid. A straight line along any row or column of the jpeg raster (or ar_mask array) will intersect different lat lon values for every pixel.  The coordinate information for our rectangular ar_mask array therefore contains lat and lon variables that are two dimensions and describe the entire grid of 1152x1152 points with unique lat/lon values for each pixel (array location) of ar_mask. The standard netcdf way of denoting a curvilinear coordinate is by creating the dimensions that define the size of the ar_mask array (x,y), adding lat/lon variables that are each dimensioned (x,y) containing the lat/lon coordinates of that point, and finally adding metadata to the ar_mask array noting that the coordinates for this variable are not the dimension variables x,y but the lat/lon variables.

In [38]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import os
from rasterio.warp import transform
import urllib.request
import xarray as xr
import glob
from netCDF4 import date2num
import datetime as dt
import cftime

### Generate template file (only needs to be done once)

In [2]:
# # Read the data
# input_path='/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-copy-sav1.tif'

# da = xr.open_rasterio(input_path)
# yval=da['y']
# ryval=np.flip(yval)
# # Compute the lon/lat coordinates with rasterio.warp.transform
# ny, nx = len(da['y']), len(da['x'])
# x, y = np.meshgrid(da['x'], ryval)
# # Rasterio works with 1D arrays
# lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
#                      x.flatten(), y.flatten())
# lon = np.asarray(lon).reshape((ny, nx))
# lat = np.asarray(lat).reshape((ny, nx))
# da.coords['lon'] = (('y', 'x'), lon)
# da.coords['lat'] = (('y', 'x'), lat)

In [3]:
# da.to_netcdf(path='/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')
# # use for just antarctic

## loop through mask files to generate converted mask files

In [29]:
tmq_dict = {2000: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc",
            2001: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312100.nc",
            2002: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312100.nc",
            2003: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312100.nc",
            2004: "prw_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312100.nc"}

ivt_dict = {2000: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001-200012.nc",
            2001: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101-200112.nc",
            2002: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201-200212.nc",
            2003: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301-200312.nc",
            2004: "windhusavi_3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401-200412.nc"}

psl_dict = {2000: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312100.nc",
            2001: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312100.nc",
            2002: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312100.nc",
            2003: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312100.nc",
            2004: "psl_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312100.nc"}

pr_dict = {2000: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200001010000-200012312359.nc",
           2001: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200101010000-200112312359.nc",
           2002: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200201010000-200212312359.nc",
           2003: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200301010000-200312312359.nc",
           2004: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200401010000-200412312359.nc",
           2005: "pr_A3hr_CAM5-1-025degree_All-Hist_est1_v1-0_run002_200501010000-200512052359.nc"}

In [241]:
for year in [2001]:  # , 2002, 2003]:
    qa_antarctic = glob.glob('/glade/u/home/tking/work/cgnet/QA_xml/h5/qa*/antarctic/netcdfs/data-{}*'.format(year))

    tmq = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/prw/{}'.format(tmq_dict[year]))
    # lat: 768 lon: 1152 time: 2919 nbnd: 2
    psl = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/psl/{}'.format(psl_dict[year]))
    # lat: 768 lon: 1152 time: 2919 nbnd: 2
    ivt = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/ivt/{}'.format(ivt_dict[year]))
    # time: 2920 bound: 2 lat: 768 lon: 1152
    pr = xr.open_dataset('/glade/scratch/tking/cgnet/high_lat_QC/pr/{}'.format(pr_dict[year]))
    # height: 1 lat: 768 lon: 1152 time: 2920 nb2: 2
    # these have different coords (although tmq and psl are same, ivt just seems to be reordered/renamed,
    # pr has an extra height dimension (that is constant) and renamed other dims)
    
    for index in range(len(qa_antarctic[:3])):

        # read in mask file
        qa_aa_ds = xr.open_dataset(qa_antarctic[index])

        # read in temp file for use in notebook
        temp = xr.open_dataset('/glade/work/tking/cgnet/polar_regridding/data-2003-04-29-02-0-sav1-rev-latlon.nc')

        # rename the dimensions of the original ar_masks variable from phony_0/1 to y and x respectively
        qa_aa_ds = qa_aa_ds.rename({'phony_dim_0':'y','phony_dim_1':'x'})

        time = qa_antarctic[index].split('/')[-1].split('data-')[1].split('.nc')[0].split('-00-2')[0]
        time_year = int(time.split('-')[0])  # or = year
        time_month = int(time.split('-')[1])
        time_day = int(time.split('-')[2])
        time_hour = 0  # all files were 00

        # determine hours past a base date

        file_for_date = qa_antarctic[index].split('/')[-1].split('data-')[1].split('.nc')[0].split('-00-2')[-1]
        if file_for_date == '':
            sample_id = 0
        else:
            sample_id = int(file_for_date[1]) # format would be _#, want #
        time_index = sample_id
        # todo: should these be different? I think we really only need sample_id?

        # to do: make this temp.time[time_index]...
        temp['time'] = date2num(dt.datetime(time_year, time_month, time_day, time_hour, 0), 'hours since 1970-01-01')
        temp['time'] = temp['time'].assign_attrs({'long_name': 'time',
                                                  'units': 'hours since 1970-01-01',
                                                  'calendar': 'noleap'})


        # append the new x,y,lat,lon variables to mask using the GeoTiff temp file
        # todo: index with [time_index,sample_id,:,:]
        temp['ar_masks'] = qa_aa_ds.ar_masks

        # add all the coordinate attributes to ar_mask
        temp['ar_masks'] = temp['ar_masks'].assign_attrs({'transform' : temp.__xarray_dataarray_variable__.transform,
                                                          'crs': temp.__xarray_dataarray_variable__.crs,
                                                          'res': temp.__xarray_dataarray_variable__.res,
                                                          'is_tiled': temp.__xarray_dataarray_variable__.is_tiled,
                                                          'nodatavals': temp.__xarray_dataarray_variable__.nodatavals,
                                                          'scales': temp.__xarray_dataarray_variable__.scales,
                                                          'offsets': temp.__xarray_dataarray_variable__.offsets,
                                                          'AREA_OR_POINT': temp.__xarray_dataarray_variable__.AREA_OR_POINT})

        temp['date'] = str(time_year)+str(time_month)+str(time_day)
        temp['date'] = temp['date'].assign_attrs({'long_name':'current date (YYYYMMDD)'})
        temp['datesec'] = str(sample_id)
        temp['datesec'] = temp['datesec'].assign_attrs({'long_name':'current seconds of current date'})

        # put in all the variable data that we received from Sol (TMQ, IVT, PSL, etc.)
        # These variables will be dimensioned (time,y,x) in your single output file and
        # have the same lat lon coordinate attribute as the ar_mask array.
        # construct the filenames for TMQ, IVT etc from the date info that you have
        # then open, read and close those files inside your processing loop.

        # open tmq file for the correct date and read tmq 
        tmq_subset = tmq.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
        psl_subset = psl.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
        ivt_subset = ivt.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
        pr_subset = pr.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest')
    
        temp['tmq'] = tmq_subset.drop('time_bnds').drop('time').to_array()[0]
        temp['psl'] = psl_subset.drop('time_bnds').drop('time').to_array()[0]
        temp['ivt'] = ivt_subset.drop('time').drop('time_bnds').drop('bounds_lat').drop_dims('bound').to_array()[0]
        temp['pr'] = pr_subset.drop('time_bnds').drop('time').to_array()[0][0]
        
        #tmq values seem to be present, other vars are missing values
        temp['tmq'] = temp['tmq'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
                                                'transform': temp['__xarray_dataarray_variable__'].transform, 
                                                'res':temp['__xarray_dataarray_variable__'].res,
                                                'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
                                                'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
                                                'scales':temp['__xarray_dataarray_variable__'].scales,
                                                'offsets': temp['__xarray_dataarray_variable__'].offsets,
                                                'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
        temp['psl'] = temp['psl'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
                                                'transform': temp['__xarray_dataarray_variable__'].transform, 
                                                'res':temp['__xarray_dataarray_variable__'].res,
                                                'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
                                                'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
                                                'scales':temp['__xarray_dataarray_variable__'].scales,
                                                'offsets': temp['__xarray_dataarray_variable__'].offsets,
                                                'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
        temp['pr'] = temp['pr'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
                                                'transform': temp['__xarray_dataarray_variable__'].transform, 
                                                'res':temp['__xarray_dataarray_variable__'].res,
                                                'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
                                                'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
                                                'scales':temp['__xarray_dataarray_variable__'].scales,
                                                'offsets': temp['__xarray_dataarray_variable__'].offsets,
                                                'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
        temp['ivt'] = temp['ivt'].assign_attrs({'crs': temp['__xarray_dataarray_variable__'].crs,
                                                'transform': temp['__xarray_dataarray_variable__'].transform, 
                                                'res':temp['__xarray_dataarray_variable__'].res,
                                                'is_tiled':temp['__xarray_dataarray_variable__'].is_tiled,
                                                'nodatavals':temp['__xarray_dataarray_variable__'].nodatavals,
                                                'scales':temp['__xarray_dataarray_variable__'].scales,
                                                'offsets': temp['__xarray_dataarray_variable__'].offsets,
                                                'AREA_OR_POINT': temp['__xarray_dataarray_variable__'].AREA_OR_POINT})
        temp = temp.drop_vars('__xarray_dataarray_variable__')

        filename = qa_antarctic[index].split('/')[-1]
        temp.to_netcdf(path='/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/'+filename)
        print('made netcdf {}'.format(filename))

        qa_aa_ds.close()

    tmq.close()
    ivt.close()
    psl.close()
    pr.close()

made netcdf data-2001-01-01-00-2.nc
made netcdf data-2001-03-01-00-2.nc
made netcdf data-2001-05-14-00-2_1.nc


In [227]:
# this seems to work, but turns to all nan's when set temp['ivt'] equal to it...
# ivt.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest').drop('time').drop('time_bnds').drop('bounds_lat').drop('bounds_lon').to_array()[0]


'+proj=stere +lat_0=-90 +lat_ts=-90 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs=True'

In [223]:
# temp['ivt'] = 
# ivt.sel(time=cftime.DatetimeNoLeap(time_year, time_month, time_day, 0, 0, 0, 0, has_year_zero=True), method='nearest').drop('time').drop('time_bnds').drop('bounds_lat').drop_dims('bound').to_array()[0]
# temp['tmq'][0]

<xarray.DataArray 'tmq' (lat: 768, lon: 1152)>
array([[0.48369142, 0.48369142, 0.48369142, ..., 0.48369142, 0.48369142,
        0.48369142],
       [0.45649925, 0.4565299 , 0.4565541 , ..., 0.45641854, 0.45644265,
        0.45646775],
       [0.45647562, 0.45649317, 0.4565069 , ..., 0.45643333, 0.45644432,
        0.45645648],
       ...,
       [5.158637  , 5.160277  , 5.161709  , ..., 5.153891  , 5.1550174 ,
        5.156895  ],
       [5.067685  , 5.06795   , 5.0682187 , ..., 5.0668416 , 5.0671234 ,
        5.067411  ],
       [5.0942717 , 5.0942717 , 5.0942717 , ..., 5.0942717 , 5.0942717 ,
        5.0942717 ]], dtype=float32)
Coordinates:
  * lon       (lon) float32 0.0 0.3125 0.625 0.9375 ... 358.8 359.1 359.4 359.7
  * lat       (lat) float32 -90.0 -89.77 -89.53 -89.3 ... 89.3 89.53 89.77 90.0
    variable  <U3 'prw'
    height    float32 0.0
Attributes: (12/21)
    forcing:                   SST=sst_HadOIBl_bc_0.23x0.31_1870_2008_c091020...
    institute_run_id:          cam5_1_amip_run2
    institution:               Lawrence Berkeley National Laboratory, Berkele...
    institute_id:              LBNL
    experiment_family:         All-Hist
    experiment:                est1
    ...                        ...
    parent_subexperiment:      N/A
    parent_run_id:             N/A
    project_id:                C20C+ Detection and Attribution Project
    license:                   Creative Commons License: http://creativecommo...
    history:                   2017-02-17T01:30:23Z: Naming and attribute con...
    creation_date:             2017-02-17T01:30:23Z

In [240]:
temp

<xarray.Dataset>
Dimensions:   (band: 3, y: 1152, x: 1152, lon: 1152, lat: 768)
Coordinates:
  * band      (band) int64 1 2 3
  * y         (y) float64 8.905e+06 8.89e+06 8.874e+06 ... -8.912e+06 -8.927e+06
  * x         (x) float64 -8.902e+06 -8.887e+06 -8.871e+06 ... 8.884e+06 8.9e+06
  * lon       (lon) float32 0.0 0.3125 0.625 0.9375 ... 358.8 359.1 359.4 359.7
  * lat       (lat) float32 -90.0 -89.77 -89.53 -89.3 ... 89.3 89.53 89.77 90.0
    variable  <U3 'prw'
    height    float32 0.0
Data variables:
    time      int64 274944
    ar_masks  (y, x) int64 ...
    date      <U7 '2001514'
    datesec   <U1 '1'
    tmq       (lat, lon) float32 0.4837 0.4837 0.4837 ... 5.094 5.094 5.094
    psl       (lat, lon) float32 1.001e+05 1.001e+05 ... 1.022e+05 1.022e+05
    ivt       (lat, lon) float32 4.925 4.925 4.925 4.925 ... 38.64 38.64 38.64
    pr        (lat, lon) float32 3.316e-09 3.316e-09 ... 1.796e-06 1.796e-06

# PART 2:

### use the standard ESMF mapping procedure to go from the projected stereographic polar mask grids to our regular gridded data from CESM. Use Steve's python code and John's mapping file commands to do this

In [35]:
# We want to do this to get masks and state information on two different grids
# need new mapping file to do this, then run through Steve's routine

In [ ]:
# Steve Yeager has a utility function for remapping CAM-SE output (see remap_camse function below):
#     https://github.com/sgyeager/mypyutils/blob/main/mypyutils/regrid_utils.py

import xarray as xr
import numpy as np
import scipy.sparse as sps
import cf_xarray

def remap_camse(ds, dsw, varlst=[]):
    #dso = xr.full_like(ds.drop_dims('ncol'), np.nan)
    dso = ds.drop_dims('ncol').copy()
    lonb = dsw.xc_b.values.reshape([dsw.dst_grid_dims[1].values, dsw.dst_grid_dims[0].values])
    latb = dsw.yc_b.values.reshape([dsw.dst_grid_dims[1].values, dsw.dst_grid_dims[0].values])
    weights = sps.coo_matrix((dsw.S, (dsw.row-1, dsw.col-1)), shape=[dsw.dims['n_b'], dsw.dims['n_a']])
    if not varlst:
        for varname in list(ds):
            if 'ncol' in(ds[varname].dims):
                varlst.append(varname)
        if 'lon' in varlst: varlst.remove('lon')
        if 'lat' in varlst: varlst.remove('lat')
        if 'area' in varlst: varlst.remove('area')
    for varname in varlst:
        shape = ds[varname].shape
        invar_flat = ds[varname].values.reshape(-1, shape[-1])
        remapped_flat = weights.dot(invar_flat.T).T
        remapped = remapped_flat.reshape([*shape[0:-1], dsw.dst_grid_dims[1].values,
                                          dsw.dst_grid_dims[0].values])
        dimlst = list(ds[varname].dims[0:-1])
        dims={}
        coords={}
        for it in dimlst:
            dims[it] = dso.dims[it]
            coords[it] = dso.coords[it]
        dims['lat'] = int(dsw.dst_grid_dims[1])
        dims['lon'] = int(dsw.dst_grid_dims[0])
        coords['lat'] = latb[:,0]
        coords['lon'] = lonb[0,:]
        remapped = xr.DataArray(remapped, coords=coords, dims=dims, attrs=ds[varname].attrs)
        dso = xr.merge([dso, remapped.to_dataset(name=varname)])
    return dso

In [ ]:
# Here is a notebook demonstrating how this is used:
#     /glade/u/home/yeager/analysis/python/toshare/CLM_field_regrid.ipynb

From John:
Here are the cheyenne commands I used to create a map file and regrid one of your new data files.  I created a source grid description (sp_stereographic_SCRIP.nc) using the ncl script in ~jet/stereoscrip.ncl. The output file ~jet/data-2003-12-20-00-2_1.regrid_f09_near.nc looks like it might be OK for a nearest neighbor interpolation but I would need to see the corresponding tmq data to make sure.  The regridded file had the dimensions x and y and I renamied those  to lon,lat so ncview could overlay a continental outline. I ran 
ncrename -dx,lon -dy,lat data-2003-12-20-00-2_1.regrid_f09_near.nc
on the file to do that renaming.

 This all could be better done with some of the python regridding that you and Katie were looking into or maybe even the python version xesmf.

module load gnu/9.1.0
module load esmf_libs/8.0.0
module load esmf-8.0.0-ncdfio-mpi-O
module load nco/4.7.9

set srcgrid=sp_stereo
set dstgrid=f09
set srcgridfile=/glade/u/home/jet/sp_stereographic_SCRIP.nc
set dstgridfile=/glade/p/cesmdata/inputdata/share/scripgrids/fv0.9x1.25_141008.nc
set srcinitfile=/glade/u/home/tking/work/cgnet/QA_xml/all_antarctic_converted_masks/data-2003-12-20-00-2_1.nc
set dstinitfile=data-2003-12-20-00-2_1.regrid_f09_near.nc

#create the map file
ESMF_RegridWeightGen --ignore_unmapped --src_regional -m neareststod -w map_${srcgrid}_to_${dstgrid}_near.nc -s ${srcgridfile} -d ${dstgridfile}

#use the mapfile to remap srcinitfile to dstinitfile
ncremap -m ./map_${srcgrid}_to_${dstgrid}_near.nc -i ${srcinitfile} -o ${dstinitfile}